In [5]:
import sys
import numpy as np 
import pandas as pd 
import random
import matplotlib.pyplot as plt 
import os
import datetime as datetime
pd.set_option('display.max_rows', 200)
%matplotlib inline

In [6]:
# define folder where data resides 
DATAFOLDER = "/Users/Colin/Google_Drive/ML_DSGA_1003/asylum_project"

## Read in Dunn Data

In [7]:
master_dunn = pd.read_csv(os.path.join(DATAFOLDER, 
                                       'raw/_decision_scheduling_merge_final_converted.csv'), 
                          encoding='latin-1', low_memory=False) # gets UnicodeDecodeError otherwise 
master_dunn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 602500 entries, 0 to 602499
Columns: 182 entries, Unnamed: 0 to last_hearing_on_comp_date
dtypes: bool(1), float64(72), int64(39), object(70)
memory usage: 832.6+ MB


## Generate Prior Percentages

In [34]:
# prior percentages for aggregate
agt_priors = master_dunn['grant'].mean()
# overall grant percent
agt_priors
agt_counts = master_dunn.groupby(['grant']).count()['idncase']
agt_counts

grant
0    388769
1    213731
Name: idncase, dtype: int64

In [27]:
# prior percentages for nationality
nat_priors = master_dunn.groupby(['nat']).mean()['grant']
nat_priors.head()
nat_counts = master_dunn.groupby(['nat', 'grant']).count()['idncase']
nat_counts

nat  grant
??   0          131
     1           67
AB   0            1
AC   0           21
     1            3
AF   0         1872
     1         3444
AG   0          611
     1          359
AL   0         5845
     1         6308
AM   0         3770
     1         3254
AN   0            8
     1            8
AO   0          224
     1          128
AR   0         1104
     1          152
AS   0           40
     1            4
AU   0           30
     1            5
AZ   0          321
     1          462
BA   0           68
     1           19
BB   0           20
     1            2
BC   0           10
     1            3
BD   0            4
     1            3
BE   0           53
     1           35
BF   0          157
     1           21
BG   0         4249
     1         2017
BH   0          174
     1           26
BI   0          274
     1          209
BL   0          366
     1          131
BM   0          634
     1         1785
BN   0           61
     1           50
BO   0   

In [28]:
# prior percentages for affirmative vs defensive
affirm_priors = master_dunn.groupby(['affirmative']).mean()['grant']
affirm_priors.head()
affirm_counts = master_dunn.groupby(['affirmative', 'grant']).count()['idncase']
affirm_counts

affirmative  grant
0.0          0        197756
             1         76467
1.0          0        180196
             1        135187
Name: idncase, dtype: int64

In [11]:
# make dt col
master_dunn['input_dt'] = master_dunn[['input_year', 'input_month', 'input_day']]\
.apply(lambda x : '{}{}{}'.format(str(x[0]).split('.')[0],str(x[1]).split('.')[0],str(x[2]).split('.')[0]), axis=1)\
.apply(pd.to_datetime, format='%Y%m%d', errors='coerce')

In [12]:
# get day of the week
master_dunn['dow'] = master_dunn['input_dt'].apply(lambda x: x.weekday())

In [25]:
# prior percentage for DOW
dow_priors = master_dunn.groupby(['dow']).mean()['grant']
dow_priors
dow_counts = master_dunn.groupby(['dow', 'grant']).count()['idncase']
dow_counts

dow  grant
0.0  0        68955
     1        36881
1.0  0        73035
     1        39326
2.0  0        68900
     1        36346
3.0  0        71852
     1        38036
4.0  0        65985
     1        34894
5.0  0         1069
     1          453
6.0  0         1456
     1          691
Name: idncase, dtype: int64

In [14]:
# round adj_time_start down
def round_down(num, divisor):
    return num - (num%divisor)

In [15]:
# round time down to the hour
# 8:59 -> 8:00 for instance
master_dunn['rounded_time'] = master_dunn['adj_time_start'].apply(lambda x: round_down(x, 100))

In [29]:
# prior percentage for time
time_priors = master_dunn.groupby(['rounded_time']).mean()['grant']
time_priors
time_counts = master_dunn.groupby(['rounded_time', 'grant']).count()['idncase']
time_counts

rounded_time  grant
800           0         70063
              1         37446
900           0         96366
              1         46455
1000          0         44127
              1         24409
1100          0          5156
              1          4874
1200          0          2897
              1          2570
1300          0        129536
              1         69498
1400          0         28337
              1         17661
1500          0          9697
              1          6752
1600          0          2242
              1          3882
1700          0            37
              1            27
1800          0           206
              1            86
1900          0           105
              1            71
Name: idncase, dtype: int64

In [30]:
# language prior
language_priors = master_dunn.groupby(['lang']).mean()['grant']
language_priors.head()
language_counts = master_dunn.groupby(['lang', 'grant']).count()['idncase']
language_counts

lang  grant
???   0         954
      1         288
AAR   1           1
ACC   0           1
ACE   0           2
      1           1
ACH   0           4
      1           3
AFE   0           1
AFR   0           5
      1           3
AGC   0          40
      1           3
AGH   1           2
AKA   0          61
      1          18
AKU   0           1
      1           2
ALB   0        7680
      1        8211
ALG   0          18
      1          12
AMD   0           2
      1          15
AMH   0        2685
      1        4537
ANU   0           2
AR    0        6330
      1        6653
ARA   0          44
      1          43
ARK   0           4
      1          22
ARM   0        4466
      1        3759
ASH   0          44
      1          10
ASY   0         131
      1         291
AZJ   0          19
      1          39
BAG   0           5
      1           3
BAI   0           3
      1           8
BAJ   0           2
      1           5
BAM   0         349
      1         393
BAN   0 

In [31]:
# lawyer prior
lawyer_priors = master_dunn.groupby(['lawyer']).mean()['grant']
lawyer_priors
lawyer_counts = master_dunn.groupby(['lawyer', 'grant']).count()['idncase']
lawyer_counts

lawyer  grant
0       0         74587
        1          7914
1       0        314182
        1        205817
Name: idncase, dtype: int64

In [19]:
def calibrate_beta_priors(aggregate_mean): 
    """
    Takes aggregate rate and return Beta priors (alpha, beta) with prior mean approximating aggregate rate, 
    with effective sample size of 10 
    """
    
    rounded_rate = np.round(aggregate_mean, 1)
    alpha = int(rounded_rate * 10) 
    beta = 10 - alpha 
    
    return alpha, beta

In [20]:
def compute_posterior_mean(alpha_prior, beta_prior, num_positives, num_total): 
    """ 
    Takes Beta priors (alpha, beta) along with observed data (num_total, num_positives) 
    and returns posterior mean 
    """
    
    updated_alpha = alpha_prior + num_positives 
    updated_beta = beta_prior + num_total - num_positives 
    
    posterior_mean = float(updated_alpha) / (updated_alpha + updated_beta)
    
    return posterior_mean

In [21]:
def get_beta_adj_rate(aggregate_mean, num_positives, num_total): 
    """ 
    Takes aggregate mean as a float (from 0 to 1), num_total (integer), and num_positives (integer) 
    and return the 'Beta-adjusted' rate. 
    Example: if in total 30% of Chinese nationality cases were granted, and a specific judge saw 20 cases 
    and granted 14 of them, input aggregate_mean=0.3, num_total=20, and num_positives=14 
    """
    
    if type(aggregate_mean) is not float: 
        raise ValueError("Please enter a float for aggregate mean!")
        
    if aggregate_mean < 0 or aggregate_mean > 1: 
        raise ValueError("Aggregate mean must be between 0 and 1!")
            
    alpha_prior, beta_prior = calibrate_beta_priors(aggregate_mean)
    posterior_mean = compute_posterior_mean(alpha_prior, beta_prior, num_positives, num_total)
    
    return posterior_mean

In [36]:
# list of percent dfs
percent_list = [agt_priors, nat_priors, affirm_priors, dow_priors, time_priors, language_priors, lawyer_priors]
# list of count dfs
count_list = [agt_counts, nat_counts, affirm_counts, dow_counts, time_counts, language_counts, lawyer_counts]